# OpenSearch Hybrid 검색을 통한 RAG
> 이 노트북은  SageMaker Studio* **`Data Science 3.0`** kernel 및 ml.t3.medium 인스턴스에서 테스트 되었습니다.
---
### 중요
- 이 노트북은 Anthropic 의 Claude-v2 모델 접근 가능한 분만 실행 가능합니다. 
- 접근이 안되시는 분은 노트북의 코드와 결과 만을 확인 하시면 좋겠습니다.
- 만일 실행시에는 **"과금"** 이 발생이 되는 부분 유념 해주시기 바랍니다.

### 선수조건
- 이 노트북은 이전 노트북인 "02_KR_RAG_OpenSearch_Claude.ipynb" 이 완료 되었다고 가정 합니다.
    - 오픈서치 인텍스 관련 정보를 참조 합니다.


### 설정

이 노트북의 나머지 부분을 실행하기 전에 아래 셀을 실행하여 (필요한 라이브러리가 설치되어 있는지 확인하고) Bedrock에 연결해야 합니다.



In [2]:
%load_ext autoreload
%autoreload 2

import sys, os
module_path = ".."
sys.path.append(os.path.abspath(module_path))

# 1. Bedrock Client 생성

In [3]:
import json
import boto3

from utils import bedrock, print_ww


# ---- ⚠️ Un-comment and edit the below lines as needed for your AWS setup ⚠️ ----

# os.environ["AWS_DEFAULT_REGION"] = "<REGION_NAME>"  # E.g. "us-east-1"
# os.environ["AWS_PROFILE"] = "<YOUR_PROFILE>"
# os.environ["BEDROCK_ASSUME_ROLE"] = "<YOUR_ROLE_ARN>"  # E.g. "arn:aws:..."
# os.environ["BEDROCK_ENDPOINT_URL"] = "<YOUR_ENDPOINT_URL>"  # E.g. "https://..."

os.environ["AWS_PROFILE"] = "bedrock_claude"

boto3_bedrock = bedrock.get_bedrock_client(
    assumed_role=os.environ.get("BEDROCK_ASSUME_ROLE", None),
    endpoint_url=os.environ.get("BEDROCK_ENDPOINT_URL", None),
    region=os.environ.get("AWS_DEFAULT_REGION", None),
)

Create new client
  Using region: us-east-1
  Using profile: bedrock_claude
  Using profile: bedrock_claude
boto3 Bedrock client successfully created!
bedrock(https://bedrock.us-east-1.amazonaws.com)


# 2. Amazon Kendra 및 LLM 인 Claude-v2 모델 로딩

## LLM 로딩 (Claude-v2)

In [4]:
from langchain.llms.bedrock import Bedrock

# - create the Anthropic Model
llm_text = Bedrock(
    model_id="anthropic.claude-v2",
    client=boto3_bedrock,
    model_kwargs={
        'max_tokens_to_sample':512
    }
)
llm_text

Bedrock(client=<botocore.client.Bedrock object at 0x7f1fa1c88a00>, region_name=None, credentials_profile_name=None, model_id='anthropic.claude-v2', model_kwargs={'max_tokens_to_sample': 512}, endpoint_url=None, cache=None, verbose=False, callbacks=None, callback_manager=None, tags=None, metadata=None)

## Embedding 모델 선택 (Kendra)

## 선수 조건
- Kendra indexing이 되어 있어야 합니다

In [16]:

# from utils.rag import KoSimCSERobertaContentHandler, SagemakerEndpointEmbeddingsJumpStart

# def get_embedding_model(is_bedrock_embeddings, is_KoSimCSERobert, is_kendra, aws_region, endpont_name=None, kendra_index_id=None):
#     if is_bedrock_embeddings:

#         # We will be using the Titan Embeddings Model to generate our Embeddings.
#         from langchain.embeddings import BedrockEmbeddings
#         llm_emb = BedrockEmbeddings(
#           client=boto3_bedrock,
#           model_id = "amazon.titan-embed-g1-text-02" # amazon.titan-e1t-medium, amazon.titan-embed-g1-text-02
#         )        

#         print("Bedrock Embeddings Model Loaded")
#     elif is_KoSimCSERobert:
#         LLMEmbHandler = KoSimCSERobertaContentHandler()
#         endpoint_name_emb = endpont_name
#         llm_emb = SagemakerEndpointEmbeddingsJumpStart(
#             endpoint_name=endpoint_name_emb,
#             region_name=aws_region,
#             content_handler=LLMEmbHandler,
#         )        
#         print("KoSimCSERobert Embeddings Model Loaded")
#     elif is_kendra:
        
#         llm_emb = AmazonKendraRetriever(
#             index_id=kendra_index_id,
#             region_name=aws_region,
#             top_k=3,
#             attribute_filter = {
#                 "EqualsTo": {      
#                     "Key": "_language_code",
#                     "Value": {
#                         "StringValue": "ko"
#                     }
#                 }
#             }
#           )
#         print("Kendra Loaded")
#     else:
#         llm_emb = None
#         print("No Embedding Model Selected")
    
#     return llm_emb

#### [중요] is_KoSimCSERobert == True 일시에 endpoint_name 을 꼭 넣어 주세요.
#### [중요] is_kendra == True 일시에 kendra_index_id 을 꼭 넣어 주세요.

In [26]:
# is_bedrock_embeddings = False
# is_KoSimCSERobert = False
# is_kendra = True

aws_region = os.environ.get("AWS_DEFAULT_REGION", None)

##############################
# Parameters for is_KoSimCSERobert
##############################
#endpont_name = "KoSimCSE-roberta-2023-09-01-14-34-11"

##############################
# Parameters for is_kendra
##############################
kendra_index_id = "078c5b9e-9062-48c1-bdb8-02b43b3da2fd"

##############################

# llm_emb = get_embedding_model(
#     is_bedrock_embeddings,
#     is_KoSimCSERobert,
#     is_kendra,
#     aws_region,
#     kendra_index_id=kendra_index_id
# )    

## 오픈 서치 도메인 및 인증 정보 세팅

- [langchain.vectorstores.opensearch_vector_search.OpenSearchVectorSearch](https://api.python.langchain.com/en/latest/vectorstores/langchain.vectorstores.opensearch_vector_search.OpenSearchVectorSearch.html)

#### [중요] 아래에 OpenSearch ID/PW 를 입력을 해주세요.

In [18]:

os.environ["OpenSearch_UserName"] = "Type ID" 
os.environ["OpenSearch_UserPassword"] = "Type PW" 


rag_user_name = os.environ["OpenSearch_UserName"]
rag_user_password = os.environ["OpenSearch_UserPassword"]


In [19]:
# OpenSearch Dashboards URL = https://search-gonsoo-jeiuxgzz4322ulsrbokvnqispu.us-east-1.es.amazonaws.com/_dashboards
opensearch_domain_endpoint = "https://search-gonsoo-jeiuxgzz4322ulsrbokvnqispu.us-east-1.es.amazonaws.com"
http_auth = (rag_user_name, rag_user_password) # Master username, Master password

index_name = "fsi-v10"


## LangChain OpenSearch VectorStore 생성

In [20]:
from langchain.vectorstores import OpenSearchVectorSearch

vectro_db = OpenSearchVectorSearch(
    index_name=index_name,
    opensearch_url=opensearch_domain_endpoint,
    embedding_function=llm_emb,
    http_auth=http_auth, # http_auth
    is_aoss =False,
    engine="faiss",
    space_type="l2"
)

## OpenSearch Client 생성

In [21]:
from utils.rag import create_aws_opensearch_client, check_if_index_exists, delete_index

aws_client = create_aws_opensearch_client(
                                     aws_region,
                                     opensearch_domain_endpoint,
                                     http_auth)



# 3. QnA

## 프로프트 템플릿 생성

In [27]:
from langchain import PromptTemplate

In [37]:
from utils.rag import create_bool_filter, run_RetrievalQA, run_RetrievalQA_kendra, show_context_used
from langchain.prompts import PromptTemplate

# prompt_template = """Human: 다음 문맥의 Information을 사용하여 고객 서비스 센터 직원처럼, 마지막 질문에 대한 목차 형식으로 답변을 제공하세요. 응답을 모르면 모른다고 말하고 응답을 만들려고 하지 마세요.

# {context}

# Question: {question}
# Assistant:"""

prompt_template = """
    
    Human: Use the following pieces of context to provide a concise answer to the question at the end. If you
    don't know the answer, just say that you don't know, don't try to make up an answer.
    
    {context}
    
    Question: {question}
    Assistant:"""



PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)


### LangChain RetrievalQA 를 통해 실행

*Atttribute filter
- https://docs.aws.amazon.com/kendra/latest/APIReference/API_AttributeFilter.html#kendra-Type-AttributeFilter-ContainsAny
- https://docs.aws.amazon.com/kendra/latest/APIReference/API_Retrieve.html

In [49]:
query = "부점장 의료비 기준 알려줘"

In [56]:
%%time
result = run_RetrievalQA_kendra(
    query=query,
    llm_text=llm_text,
    PROMPT=PROMPT,
    kendra_index_id=kendra_index_id,
    k=5,
    aws_region=aws_region,
    verbose=False
)

print("##################################")
print("query: ", query)
print("##################################")
print_ww(result['result'])
show_context_used(result['source_documents'])        


##################################
query:  부점장 의료비 기준 알려줘
##################################
 문서 내용을 보면 업무 중 사고/재해 발생 시 부점장도 의료비 지원을 받을 수 있습니다.

요약하면 다음과 같습니다:

- 대상자: 재직 중인 부점장으로 업무 중 사고나 재해 발생 시
- 병가 지원: 진단서 상 가료기간 내 유급병가 지원
- 의료비 지원: 사고로 인한 의료비 지원
- 지급: 결재 완료 후 월 3회 지급일에 개인 경비계좌로 입금
- 신청: 인사담당자에게 등기로 신청서류 제출 (진단서, 의료비 서류 등)

따라서 부점장도 업무 중 사고나 재해 시 병가와 의료비 지원을 받을 수 있습니다.
-----------------------------------------------
1. Chunk: 726 Characters
-----------------------------------------------
Document Title: (PPT원본) 202302_복리후생운영기준_230918.pptx
Document Excerpt:
‹#› | 2-3. 의료비 지원 (업무상) 업무 중의 사고/재해 발생시 의료비와 병가를 지원합니다. 지원 기준 대상자 병가 지원 의료비 지원 지급 재직 중인 파트너로서 업무 중
사고 및 재해시 진단서 상 가료기간 내 유급 병가 부여 ※ EP 근태품의 상신 요함 (진단서, 사건사고보고서 첨부) 해당 사고로 인해 발생하는 의료비 지원 ※ 인사담당자에게 등기
송부하여 신청 (진단서 및 의료비 서류) 결재완료 시점에 따라 월 3회 지급일자에 개인경비계좌로 지급 (10, 20, 말일) [사건사고보고서 작성] ▶ 업무상 사고/재해 관련
경위를 사건사고보고 등록합니다. - 작성 위치 : 블라섬 > 업무시스템 > 리스크관리시스템(RMS) > 사건사고보고 등록 (파트너 개인정보는 제외하고, 육하원칙에 따라 상세히
작성) - 작성자 : 사건 당시의 시프

In [ ]:
print (f'question: {query}')
print (f'response: {response}')